In [2]:
from os import path
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd
from time import sleep

METADATA_DESTINATION = '../dataset/imdb_crop/'
METADATA_CROPPED_FILE_MATLAB = METADATA_DESTINATION + 'imdb.mat'

In [2]:
'''
# RETRIEVING NAMES AND IMG URLS
names = open('names.html', 'r', errors='ignore')
actors = open("most_famous_actors.txt", 'w')

actors_names = []
img_urls = []
soup = BeautifulSoup(names, 'html.parser')
for span in soup.findAll('span'):
    for img in span.findAll('img'):
        if img:
            actors_names.append(img.attrs['alt'])
            img_urls.append(re.split(r' |,', img.attrs['srcset'])[-2])

for i in range(len(actors_names)):
    actors.write(actors_names[i] + ';' + img_urls[i] + '\n')
'''

'\n# RETRIEVING NAMES AND IMG URLS\nnames = open(\'names.html\', \'r\', errors=\'ignore\')\nactors = open("most_famous_actors.txt", \'w\')\n\nactors_names = []\nimg_urls = []\nsoup = BeautifulSoup(names, \'html.parser\')\nfor span in soup.findAll(\'span\'):\n    for img in span.findAll(\'img\'):\n        if img:\n            actors_names.append(img.attrs[\'alt\'])\n            img_urls.append(re.split(r\' |,\', img.attrs[\'srcset\'])[-2])\n\nfor i in range(len(actors_names)):\n    actors.write(actors_names[i] + \';\' + img_urls[i] + \'\n\')\n'

In [3]:
actors = open("most_famous_actors.txt", 'r')
actors = actors.read()
actors_names = re.split(r';|\n', actors)[::2]
actors_names = actors_names[:-1]
actors_img_urls = re.split(r';|\n', actors)[1::2]

In [4]:
'''
# MOST FAMOUS ACTORS IN IMDB
head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'imdb.pickle')
df = pd.read_pickle(path_serialized)
df.head()

df_actors_imdb = pd.DataFrame(columns=['name', 'url_img', 'gender', 'age'])
with tqdm(total=len(actors_names)) as pbar:
        for i in range(len(actors_names)):
            q = 'name=="' + actors_names[i] + '"'
            query_results = df.query(q)[["age","gender"]]
            if not query_results.empty:
                new_row = query_results.iloc[0]
                new_row['name'] = actors_names[i]
                new_row['url_img'] = actors_img_urls[i]
                df_actors_imdb = df_actors_imdb.append(new_row)
            pbar.update(1)
df_actors_imdb.reset_index()

len(df_actors_imdb)


head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'imdb_most_famous_actors.pickle')
#df_actors_imdb.to_pickle(path_serialized)

df_actors_imdb = pd.read_pickle(path_serialized)
df_actors_imdb.head()

df_actors_imdb.describe(include='all')
# female 0, male 1
'''

'\n# MOST FAMOUS ACTORS IN IMDB\nhead, tail = path.split(METADATA_CROPPED_FILE_MATLAB)\npath_serialized = path.join(head, \'imdb.pickle\')\ndf = pd.read_pickle(path_serialized)\ndf.head()\n\ndf_actors_imdb = pd.DataFrame(columns=[\'name\', \'url_image\', \'gender\', \'age\'])\nwith tqdm(total=len(actors_names)) as pbar:\n        for i in range(len(actors_names)):\n            q = \'name=="\' + actors_names[i] + \'"\'\n            query_results = df.query(q)[["age","gender"]]\n            if not query_results.empty:\n                new_row = query_results.iloc[0]\n                new_row[\'name\'] = actors_names[i]\n                new_row[\'url_img\'] = actors_img_urls[i]\n                df_actors_imdb = df_actors_imdb.append(new_row)\n            pbar.update(1)\ndf_actors_imdb.reset_index()\n\nlen(df_actors_imdb)\n\n\nhead, tail = path.split(METADATA_CROPPED_FILE_MATLAB)\npath_serialized = path.join(head, \'imdb_most_famous_actors.pickle\')\n#df_actors_imdb.to_pickle(path_serialized

In [5]:
'''
# MOST FAMOUS ACTORS IN WIKIDATA
from datetime import datetime
from dateutil.relativedelta import relativedelta

# not so inclusive :/
gender_dict = {'http://www.wikidata.org/entity/Q6581097': 1,
               'http://www.wikidata.org/entity/Q6581072': 0,
               'http://www.wikidata.org/entity/Q2449503': 1,
               'https://www.wikidata.org/wiki/Q1052281': 0}

def do_query(an, aiu):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
    gender_dict = {'http://www.wikidata.org/entity/Q6581097': 1, 'http://www.wikidata.org/entity/Q6581072': 0 }
    df_actors_wiki = pd.DataFrame(columns=['name', 'url_img', 'gender', 'age'])
    with tqdm(total=len(an)) as pbar:
        for i in range(len(an)):
            q = "SELECT  DISTINCT ?gender (SAMPLE(?birth) as ?birth_date)" \
            " (SAMPLE(?death_date) as ?dateOfDeath)  WHERE {" \
            "?id wdt:P31 wd:Q5." \
            "?id wdt:P569 ?birth ." \
            "?id wdt:P21 ?gender." \
            "OPTIONAL{?id wdt:P570 ?death_date .}" \
            "SERVICE wikibase:label {" \
            "bd:serviceParam wikibase:language 'en'." \
            "?id rdfs:label ?idLabel ." \
            "}?id ?label\""  + an[i] + \
            "\"@en." \
            "}" \
            "GROUP BY ?gender"
            sparql.setQuery(q)
            sparql.setReturnFormat(JSON)
            results = sparql.query().convert()
            results_df = pd.io.json.json_normalize(results['results']['bindings'])
            if not results_df.empty and len(results_df) == 1:
                if results_df["gender.value"].iloc[0] in gender_dict.values():
                    gender = gender_dict[results_df["gender.value"].iloc[0]]
                    birth_date = datetime.strptime(results_df["birth_date.value"].iloc[0],"%Y-%m-%dT%H:%M:%SZ")
                    age = relativedelta(datetime.now(), birth_date).years
                    new_row = {"name": an[i], "url_img": aiu[i], "gender": gender, "age": age}
                    df_actors_wiki = df_actors_wiki.append(new_row, ignore_index=True)
            pbar.update(1)
    df_actors_wiki.reset_index()
    return df_actors_wiki

# to handle 'Too Many Requests' errors
df1 = do_query(actors_names[:200], actors_img_urls[:200])
sleep(60)
df2 = do_query(actors_names[200:400], actors_img_urls[200:400])
sleep(60)
df3 = do_query(actors_names[400:600], actors_img_urls[400:600])
sleep(60)
df4 = do_query(actors_names[600:800], actors_img_urls[600:800])
sleep(60)
df5 = do_query(actors_names[800:1000], actors_img_urls[800:1000])
sleep(60)
df6 = do_query(actors_names[1000:1200], actors_img_urls[1000:1200])
sleep(60)
df7 = do_query(actors_names[1200:1400], actors_img_urls[1200:1400])
sleep(60)
df8 = do_query(actors_names[1400:], actors_img_urls[1400:])
df_actors_wiki = df1
df_actors_wiki = pd.concat([df_actors_wiki, df2])
df_actors_wiki = pd.concat([df_actors_wiki, df3])
df_actors_wiki = pd.concat([df_actors_wiki, df4])
df_actors_wiki = pd.concat([df_actors_wiki, df5])
df_actors_wiki = pd.concat([df_actors_wiki, df6])
df_actors_wiki = pd.concat([df_actors_wiki, df7])
df_actors_wiki = pd.concat([df_actors_wiki, df8])
df_actors_wiki.head()

head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized = path.join(head, 'wiki_most_famous_actors.pickle')
#df_actors_wiki.to_pickle(path_serialized)

df_actors_wiki = pd.read_pickle(path_serialized)
df_actors_wiki.head()

df_actors_wiki.describe(include='all')
# female 0, male 1
'''

In [3]:
# COMPARISON
head, tail = path.split(METADATA_CROPPED_FILE_MATLAB)
path_serialized_wiki = path.join(head, 'wiki_most_famous_actors.pickle')
path_serialized_imdb = path.join(head, 'imdb_most_famous_actors.pickle')
df_actors_imdb = pd.read_pickle(path_serialized_imdb)
df_actors_wiki = pd.read_pickle(path_serialized_wiki)

In [8]:
print('Length IMDB: ' + str(len(df_actors_imdb)) + ', length WIKI: ' + str(len(df_actors_wiki)))

Length IMDB: 1245, length WIKI: 969


In [ ]:
filtered_data

In [ ]:
filtered_data

In [24]:
filtered_data

,name,gender,age,url_img
102,Elle Fanning,0,23,https://d3hjzzsa8cr26l.cloudfront.net/e6f09cc1...
148,Zendaya,0,25,https://d3hjzzsa8cr26l.cloudfront.net/fe3a1254...
91,Sophie Turner,0,25,https://d3hjzzsa8cr26l.cloudfront.net/6579d712...
174,Maisie Williams,0,24,https://d3hjzzsa8cr26l.cloudfront.net/49381f7d...
